<font size="8"> **Adding environmental data from ACCESS-OM2-01 to unique background points** </font>  
In this notebook, we will extract environmental data from the ACCESS-OM2-01 model outputs and add it to our data frame containing unique background points matching the spatial bias of crabeater data (see `01_Bio_data/04_Generating_background_samples.R` for more information).

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/39193/status,
Dashboard: /proxy/39193/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34685,Workers: 4
Dashboard: /proxy/39193/status,Total threads: 12
Started: Just now,Total memory: 48.00 GiB
Comm: tcp://127.0.0.1:43831,Total threads: 3
Dashboard: /proxy/34955/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:39679,


# Loading unique crabeater seal observations data frame

In [24]:
#Loading dataset as pandas data frame
bg_path = '../Biological_Data/BG_points/unique_background_20x_obs_grid.csv'
crabeaters = pd.read_csv(bg_path)

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,date,year,month,longitude,latitude,xt_ocean,yt_ocean,season_year,life_stage,decade,sector,zone,presence
0,1999-12,1999,12,85.55,-64.15,85.55,-64.157,summer,weaning,1990,East Indian,Antarctic,0
1,1999-12,1999,12,75.45,-66.15,75.45,-66.156,summer,weaning,1990,Central Indian,Antarctic,0
2,1999-12,1999,12,142.85,-64.55,142.85,-64.547,summer,weaning,1990,Central Indian,Antarctic,0
3,1999-12,1999,12,109.25,-64.75,109.25,-64.761,summer,weaning,1990,Central Indian,Antarctic,0
4,1999-12,1999,12,102.45,-64.55,102.45,-64.547,summer,weaning,1990,Central Indian,Antarctic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35561,1998-11,1998,11,74.95,-65.55,74.95,-65.565,autumn,weaning,1990,Central Indian,Antarctic,0
35562,1987-11,1987,11,108.05,-62.95,108.05,-62.955,autumn,weaning,1980,Central Indian,Antarctic,0
35563,1997-12,1997,12,91.25,-62.85,91.25,-62.864,summer,weaning,1990,Central Indian,Antarctic,0
35564,1998-11,1998,11,88.55,-64.45,88.55,-64.461,autumn,weaning,1990,Central Indian,Antarctic,0


# Adding values for static variables only
Static variables referred to any physical variables that do not change over time (at least not during the time period of our interest). Examples include depth of the water column and distance to coastline. Given that we only have one value for these variables, the process of extracting data is relatively simple. We do not need to take into account the date observations were collected.

## Defining dictionary with information about static variables
This dictionary contains the column labels for each and the name of the files for each static variable to be included in our analysis. We will also define a variable containing the full path to the folder where all static variables are stored.

In [25]:
#Full path to static variables
base_dir_static = '/g/data/v45/la6889/Chapter2_Crabeaters/Static_Variables/'

#List of static variables
varDict = {'bottom_slope_deg': 'bathy_slope_GEBCO_2D.nc',
           'dist_shelf_km': 'distance_shelf.nc',
           'dist_coast_km': 'distance_coastline.nc',
           'depth_m': 'bathy_GEBCO_2D.nc'}

## Extracting data for each observation and adding it to a new column in crabeater data

In [26]:
#Getting coordinates from crabeater dataset
lat = xr.DataArray(crabeaters.latitude.values)
lon = xr.DataArray(crabeaters.longitude.values)

#Looping through dictionary keys
for var in varDict:
    #Creating full path to file of interest
    file_path = os.path.join(base_dir_static, varDict[var])
    #Load as raster
    ras = xr.open_dataarray(file_path)
    rename_var = ras.name
    #Extracting values
    ras_sub = ras.sel(xt_ocean = lon, yt_ocean = lat, method = 'nearest')
    #Turning into data frame and rounding all columns to 3 decimal places
    ras_df = ras_sub.to_dataframe().round(3).rename(columns = {rename_var: var})
    #Adding to crabeater observations data frame
    crabeaters = pd.merge(crabeaters, ras_df, on = ['yt_ocean', 'xt_ocean'], how = 'left', sort = True).drop_duplicates()
    
#Checking results
crabeaters

,date,year,month,longitude,latitude,xt_ocean,yt_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,1999-12,1999,12,75.05,-69.25,75.05,-69.239,summer,weaning,1990,East Indian,Antarctic,0,89.812,504.080,140.256,777.867004
1,1999-12,1999,12,75.85,-69.25,75.85,-69.239,summer,weaning,1990,Central Indian,Antarctic,0,89.953,506.158,143.956,503.000000
2,1999-12,1999,12,76.05,-69.25,76.05,-69.239,summer,weaning,1990,East Indian,Antarctic,0,89.949,507.046,145.997,522.900024
3,1999-12,1999,12,77.25,-69.25,77.25,-69.239,summer,weaning,1990,Central Indian,Antarctic,0,NaN,509.341,166.208,NaN
4,1999-12,1999,12,73.95,-69.15,73.95,-69.155,summer,weaning,1990,Central Indian,Antarctic,0,NaN,494.995,130.925,599.817017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104015,1999-12,1999,12,69.25,-59.45,69.25,-59.442,summer,weaning,1990,Central Indian,Antarctic,0,89.833,556.221,688.617,4624.497070
104016,1999-12,1999,12,71.95,-59.45,71.95,-59.442,summer,weaning,1990,East Indian,Antarctic,0,89.653,536.648,699.380,4429.308105
104017,1987-11,1987,11,70.85,-59.35,70.85,-59.340,autumn,weaning,1980,Central Indian,Antarctic,0,89.980,551.535,703.061,4767.583984
104018,1998-11,1998,11,71.55,-59.35,71.55,-59.340,autumn,weaning,1990,Central Indian,Antarctic,0,89.930,548.571,707.330,4608.485840


## Saving data frame with static variables
Given that the dynamic variables take some time to extract. We will save intermediary results to avoid having to extract them again.

In [27]:
crabeaters.to_csv(bg_path, index = False)

# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [190]:
crabeaters = pd.read_csv('../Biological_Data/BG_points/unique_background_5x_obs_grid.csv')
#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)
crabeaters

,date,year,month,longitude,latitude,xt_ocean,yt_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,1999-12,1999,12,73.85,-69.25,73.85,-69.239,summer,weaning,1990,East Indian,Antarctic,0,NaN,504.582,140.256,NaN
1,1987-11,1987,11,71.65,-69.15,71.65,-69.155,autumn,weaning,1980,Central Indian,Antarctic,0,NaN,504.302,156.205,NaN
2,1987-11,1987,11,79.05,-69.15,79.05,-69.155,autumn,weaning,1980,Central Indian,Antarctic,0,NaN,506.018,204.310,NaN
3,1997-12,1997,12,74.55,-69.05,74.55,-69.070,summer,weaning,1990,Central Indian,Antarctic,0,89.715,485.126,121.468,797.050
4,1999-12,1999,12,75.35,-69.05,75.35,-69.070,summer,weaning,1990,East Indian,Antarctic,0,89.702,485.826,122.080,729.212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10774,1987-11,1987,11,69.85,-60.75,69.85,-60.739,autumn,weaning,1980,Central Indian,Antarctic,0,89.961,408.352,544.668,4415.736
10775,1988-12,1988,12,112.95,-60.75,112.95,-60.739,summer,weaning,1980,Central Indian,Antarctic,0,89.727,228.142,294.787,4340.138
10776,1999-12,1999,12,72.85,-60.15,72.85,-60.147,summer,weaning,1990,Central Indian,Antarctic,0,89.973,458.773,632.560,3646.319
10777,1999-12,1999,12,71.05,-59.55,71.05,-59.544,summer,weaning,1990,East Indian,Antarctic,0,89.971,527.919,681.598,4711.166


## Accessing ACCESS-OM2-01 model outputs
We will create a new `cosima cookbook` session to load the model outputs of interest, and we will also create a dictionary that contains useful information related to data extraction.

In [4]:
#Creating new COSIMA cookbook session
session = cc.database.create_session()

#Creating dictionary with useful information
varDict = {'model': 'ACCESS-OM2-01',
           #ACCESS-OM2-01 cycle 4 (1958-2018)
           'exp': '01deg_jra55v140_iaf_cycle4',
           #ACCESS-OM2-01 cycle 4 extension (2018-2022)
           'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
           #Temporal resolution
           'freq': '1 monthly',
           #Output folder
           'base_out': '../Environmental_Data/ACCESS-OM2-01'}

## Loading data frame with ACCESS-OM2-01 outputs
We can use this data frame to find the variable names for the environmental factors that we know are influential for the distribution of crabeater seals.

In [102]:
#Loading data frame with model outputs
# var_acc = cc.querying.get_variables(session, experiment = varDict['exp_ext'], frequency = '1 monthly')

#Searching data frame for variables of interest
var_acc[var_acc.name.str.contains('salt')]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
58,fsalt_ai_m,salt flux ice to ocean,kg/m^2/s,1 monthly,output1008/ice/OUTPUT/iceh.2023-03.nc,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
59,fsalt_m,salt flux ice to ocn (cpl),kg/m^2/s,1 monthly,output1008/ice/OUTPUT/iceh.2023-03.nc,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
96,salt,Practical Salinity,psu,1 monthly,output1008/ocean/ocean-3d-salt-1-monthly-mean-...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
97,salt_xflux_adv,rho*dzt*dyt*u*tracer,kg/sec,1 monthly,output1008/ocean/ocean-3d-salt_xflux_adv-1-mon...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
98,salt_yflux_adv,rho*dzt*dxt*v*tracer,kg/sec,1 monthly,output1008/ocean/ocean-3d-salt_yflux_adv-1-mon...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
105,sfc_salt_flux_coupler,sfc_salt_flux_coupler: flux from the coupler,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_couple...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
106,sfc_salt_flux_ice,sfc_salt_flux_ice,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_ice-1-...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
107,sfc_salt_flux_restore,sfc_salt_flux_restore: flux from restoring term,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_restor...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
120,surface_salt,Practical Salinity,psu,1 monthly,output1008/ocean/ocean-2d-surface_salt-1-month...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00


## Completing dictionary with useful variables
Now that we identified the correct name for the variable of our interest, we can complete our dictionary.

In [261]:
#Variable name in the model
varDict['var_mod'] = 'dist_km'
#Name of column where we will store the extracted data
varDict['var_short_name'] = 'dist_ice_edge_km'
#Defining if this variable is related to sea ice or not
varDict['ice_data'] = False
#Checking final dictionary
varDict

{'model': 'ACCESS-OM2-01',
 'exp': '01deg_jra55v140_iaf_cycle4',
 'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
 'freq': '1 monthly',
 'base_out': '../Environmental_Data/ACCESS-OM2-01',
 'var_mod': 'dist_km',
 'var_short_name': 'dist_ice_edge_km',
 'ice_data': False}

## Loading data from ACCESS-OM2-01
ACCESS-OM2-01 has four different cycles available with a temporal range of 60 years (1958-2018), but only cycle 4 includes biogeochemical (BGC) outputs. Since we are interested in examining the effect of some BGC variables on crabeater seals, we will use the fourth cycle in this project. Another special feature of cycle 4 is that it was extended until December 2022. Outputs from cycle 4 are available through two experiments: `01deg_jra55v140_iaf_cycle4` which goes from 1958 to 2018, and `01deg_jra55v140_iaf_cycle4_jra55v150_extension` that includes outputs from 2019 to 2022. The crabeater dataset has temporal range between 1978 and 2022, and this is the reason why we are using the two experiment of cycle 4.  
  
In the chunk below, we load the ACCESS-OM2-01 data, correct longitudes so they range between -180 and +180, and apply a transformation to temperature outputs (`temp`) only because the original units are in Kelvin and we need them in $^{\circ}C$.

In [225]:
#Loading data from fourth cycle (temporal range 1958 to 2018)
var_df = uf.getACCESSdata_SO(varDict['var_mod'], '1978-01', '2019-01', 
                              freq = varDict['freq'], ses = session, minlat = -80, maxlat = -40,
                              exp = varDict['exp'], ice_data = varDict['ice_data'])
#Loading data from fourth cycle extension (2019 to 2022)
var_df_ext = uf.getACCESSdata_SO(varDict['var_mod'], '2019-01', '2023-01', 
                              freq = varDict['freq'], ses = session, minlat = -80, maxlat = -40,
                              exp = varDict['exp_ext'], ice_data = varDict['ice_data'])

#Concatenating both data arrays into one
var_df = xr.concat([var_df, var_df_ext], dim = 'time')

#Transforming longitudes so their range is +/-180 degrees
var_df = uf.corrlong(var_df)

#Selecting Indian sector
var_df = var_df.sel(xt_ocean = slice(30, 170))

#If temperature data, transform from Kelvin to degrees C
if var_df.name == 'temp':
    var_df = var_df-273.15

#Removing duplicate variable
del var_df_ext

## *Optional: Subsetting surface layer data*
For some ocean variables, we need to subset data to extract surface values or bottom values. Subsetting data for the surface layer is an easy process, we simply need to select the first depth bin available. The `st_ocean` dimension contains the depth bins.

In [209]:
#Selecting the first depth available in the model (i.e. surface layer)
var_df = var_df.isel(st_ocean = 0)
#Removing depth dimension
var_df = var_df.squeeze().drop('st_ocean')
#Checking results - dataset has three dimensions instead of the original four
var_df

<xarray.DataArray 'temp' (time: 541, yt_ocean: 781, xt_ocean: 1400)>
dask.array<getitem, shape=(541, 781, 1400), dtype=float32, chunksize=(1, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 30.05 30.15 30.25 30.35 ... 169.8 169.9 170.0
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -40.18 -40.11 -40.03
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00

## *Optional: Subsetting bottom data*
Subsetting data for the bottom layer is not as straightforward as for the surface. This is because the bathymetry is not the same across the Southern Ocean. To identify the correct depth bin that contains the deepest values for a particular grid cell.  
  
We have included a function called `extract_bottom_layer` in the `UsefulFunctions.py` script that extracts data for the bottom layer of any variables with a depth dimension (`st_ocean`). For more details refer to the script.  

In [226]:
var_df = uf.extract_bottom_layer(var_df)
#Checking results
var_df

<xarray.DataArray 'salt' (time: 541, yt_ocean: 781, xt_ocean: 1400)>
dask.array<transpose, shape=(541, 781, 1400), dtype=float64, chunksize=(1, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 30.05 30.15 30.25 30.35 ... 169.8 169.9 170.0
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -40.18 -40.11 -40.03
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00

## Loading other dynamic variables derived from calculations or regridded
These variables are not directly available in ACCESS-OM2-01, but they have been calculated from model outputs. Refer to folder `02_Environmental_Data` to see the full details of each calculation.

In [262]:
file_path = '/g/data/v45/la6889/Chapter2_Crabeaters/SeaIce/Distance_Edge/*.nc'
var_name = varDict['var_mod']

In [263]:
#Load data
var_df = xr.open_mfdataset(sorted(glob(file_path)))[var_name]
#If mask is present as dimension, drop it
if 'mask' in var_df.coords:
    var_df = var_df.squeeze().drop('mask')
#Check results
var_df

<xarray.DataArray 'dist_km' (time: 541, yt_ocean: 713, xt_ocean: 3600)>
dask.array<concatenate, shape=(541, 713, 3600), dtype=float64, chunksize=(1, 713, 3600), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1977-12-31T12:00:00 ... 2022-12-31T12:00:00
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Extracting environmental data
We will use the `latitude` and `longitude` columns together with the `event_date` column from the crabeater seal observations to find the corresponding grid cell in the model outputs and extract the value of the environmental factor of our interest.

In [264]:
#Getting coordinates from the crabeater data
lat = xr.DataArray(crabeaters.latitude)
lon = xr.DataArray(crabeaters.longitude)
#Getting data of observation from the crabeater data
time = xr.DataArray(crabeaters.apply(lambda x: pd.to_datetime(f'{x.date}-16'), axis = 1))

## Extracting data

In [265]:
#Extracting data
var_sub = var_df.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')

#Transforming to data frame
var_pd = var_sub.to_dataframe().sort_values(['time', 'xt_ocean', 'yt_ocean'])
#Adding year and month
var_pd['year'] = var_pd.time.dt.year
var_pd['month'] = var_pd.time.dt.month
#Removing time column that is no longer needed
var_pd.drop(columns = 'time', inplace = True)
#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Renaming variable to be added to crabeater data
var_pd.rename(columns = {varDict['var_mod']: varDict['var_short_name']}, inplace = True)
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_short_name']).columns.tolist()

#Checking results
print(cols); var_pd

['yt_ocean', 'xt_ocean', 'year', 'month']


,yt_ocean,xt_ocean,dist_ice_edge_km,year,month
dim_0,,,,,
9662,-63.762,76.05,325.720123,1981,12
213,-67.043,76.15,690.285745,1981,12
167,-67.170,76.45,705.065718,1981,12
3328,-64.846,76.95,449.988341,1981,12
1752,-65.142,78.85,502.974930,1981,12
...,...,...,...,...,...
8349,-64.070,135.95,176.057370,2013,12
6879,-64.244,136.95,206.443250,2013,12
3795,-64.846,141.55,335.567183,2013,12


## Joining environmental data frame with background points
We will use the grid cell coordinates and dates to perform this join.

In [266]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters = crabeaters.drop_duplicates()
crabeaters

,date,year,month,longitude,latitude,xt_ocean,yt_ocean,season_year,life_stage,decade,...,SST_degC,bottom_temp_degC,SSS_psu,bottom_sal_psu,vel_lat_surf_msec,vel_lat_bottom_msec,vel_lon_surf_msec,vel_lon_bottom_msec,lt_pack_ice,dist_ice_edge_km
0,1999-12,1999,12,73.85,-69.25,73.85,-69.239,summer,weaning,1990,...,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,999.986663
1,1987-11,1987,11,71.65,-69.15,71.65,-69.155,autumn,weaning,1980,...,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,968.626171
2,1987-11,1987,11,79.05,-69.15,79.05,-69.155,autumn,weaning,1980,...,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,1017.552702
3,1997-12,1997,12,74.55,-69.05,74.55,-69.070,summer,weaning,1990,...,-0.287903,-1.865906,33.698875,34.609367,0.075228,0.035569,0.068277,0.031910,0.654762,1011.831253
4,1999-12,1999,12,75.35,-69.05,75.35,-69.070,summer,weaning,1990,...,-1.583099,-1.868744,33.992184,34.599453,-0.001448,0.001709,0.000258,0.022774,0.630952,1012.421868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12222,1987-11,1987,11,69.85,-60.75,69.85,-60.739,autumn,weaning,1980,...,-1.440918,-0.525208,33.781956,34.633362,-0.027406,0.025553,0.025124,0.003854,0.000000,57.178165
12223,1988-12,1988,12,112.95,-60.75,112.95,-60.739,summer,weaning,1980,...,1.761047,-0.273041,33.821957,34.664806,-0.168507,-0.039878,0.016107,-0.012456,0.000000,350.553734
12224,1999-12,1999,12,72.85,-60.15,72.85,-60.147,summer,weaning,1990,...,-1.242035,-0.331696,33.736217,34.643547,0.020679,0.003697,0.042847,-0.001025,0.011905,51.379618
12225,1999-12,1999,12,71.05,-59.55,71.05,-59.544,summer,weaning,1990,...,-0.721710,-0.528992,33.638538,34.632473,0.073491,0.002979,0.194970,0.010461,0.000000,28.631353


## Saving data frame to disk

In [267]:
#Ensure output folder exists
os.makedirs(varDict['base_out'], exist_ok = True)

#Create file path where data will be saved
file_out = os.path.join(varDict['base_out'], 'unique_background_5x_obs_all_env.csv')

#Saving as csv file
crabeaters.to_csv(file_out, index = False)